In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##### import data

In [ ]:
# tsv = tab-separated values
# quoting = 3, treats quotes as normal chars and not delimiters
df = pd.read_csv('../data/Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)

In [ ]:
df

##### clean texts

In [ ]:
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [ ]:
nltk.download('stopwords')

# lemmatization is better but slower than stemming
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
corpus = []

ps = PorterStemmer()

lz = WordNetLemmatizer()

stop_words = set(stopwords.words('english')) # set has faster lookup than list
stop_words.remove('not')

In [ ]:
for review in df['Review']:
    review = contractions.fix(review) # can't -> cannot, won't -> will not

    review = re.sub('[^a-zA-Z]', ' ', review) # non-alphabets to space

    review = review.lower()

    review = review.split() # tokenize

    # review = [ps.stem(word) for word in review if word not in stop_words]

    review = [lz.lemmatize(word) for word in review if word not in stop_words]

    review = ' '.join(review) # untokenize

    corpus.append(review)

In [ ]:
print(corpus)

##### create bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# by giving no argument, the total no. of words (len(x[0])) can be known
# then reduce the words to most frequent 'n' words
cv = CountVectorizer(max_features = 1700)

x = cv.fit_transform(corpus).toarray() # convert sparse matrix to nd array

y = df.iloc[:, -1].to_numpy()

In [ ]:
len(x[0])

##### split the data

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

##### train svm

In [ ]:
from sklearn.svm import SVC

classifier = SVC(kernel = 'linear')

classifier.fit(x_train, y_train)

In [ ]:
y_pred = classifier.predict(x_test)

##### evaluate model

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

cm = confusion_matrix(y_test, y_pred)

accuracy = accuracy_score(y_test, y_pred) # class balance
f1 = f1_score(y_test, y_pred) # class imbalance

print('confusion matrix: \n', cm)
print('\naccuracy:', accuracy)
print('f1_score:', f1)

___

##### predict on a review

In [ ]:
review = "I love this place"

In [ ]:
review = contractions.fix(review)

review = re.sub('[^a-zA-Z]', ' ', review)

review = review.lower()

review = review.split()

# review = [ps.stem(word) for word in review if word not in stop_words] 

review = [lz.lemmatize(word) for word in review if word not in stop_words]

review = ' '.join(review)

In [ ]:
print(review)

In [ ]:
review = cv.transform([review]).toarray()

In [ ]:
print(classifier.predict(review))